# 实验 2：使用 Pandas 进行基础金融数据分析

## 学习目标
在本实验结束时，你将能够：
- 使用 pandas 加载和操作真实的金融时间序列数据
- 为 OHLCV（开盘价、最高价、最低价、收盘价、成交量）数据使用适当的日期时间索引
- 使用 .info()、.describe()、.head() 和 .tail() 进行基础数据探索
- 应用基于时间的索引和切片操作（使用 DatetimeIndex）
- 计算简单的金融指标：收益率、移动平均线和基础波动率
- 使用前向填充和插值方法处理缺失数据
- 使用基础时间序列操作：重采样、滚动窗口和位移
- 在股票之间进行简单的相关性分析
- 将结果导出为 CSV 和 Excel 格式

## 介绍
本实验将使用真实的金融数据，向你介绍必要的 pandas 技能。你将学习基础的数据操作技术，这些技术构成了金融分析的基础，使用的是来自苹果、微软、谷歌和特斯拉等主要公司的真实股市数据。

## 基础概念

在深入代码之前，让我们先了解将要使用的关键概念：

### 1. OHLCV 数据结构

金融时间序列数据通常包括：
- **开盘价**: 交易期间的开盘价格
- **最高价**: 交易期间的最高价格
- **最低价**: 交易期间的最低价格
- **收盘价**: 交易期间的收盘价格
- **成交量**: 交易的股票数量

### 2. 简单收益率

**百分比变化公式：**
```
收益率 = (当前价格 - 前一期价格) / 前一期价格 × 100
```

在 pandas 中：`df['Close'].pct_change() * 100`

### 3. 移动平均线

**简单移动平均线：**
```
简单移动平均线 = 过去 N 个周期的平均值
```

在 pandas 中：`df['Close'].rolling(window=N).mean()`

### 4. 基础波动率

**收益率的标准差：**
```
波动率 = 一段时间内收益率的标准差
```

在 pandas 中：`df['Close'].pct_change().rolling(window=N).std()`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import openpyxl
import warnings
warnings.filterwarnings('ignore')

# Set display options for better output formatting
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 6)
pd.set_option('display.float_format', '{:.4f}'.format)

# Set style for better plots
plt.style.use('default')

print("📊 Basic Pandas Environment Ready")
print(f"📈 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print("=" * 50)

---

## 练习 1：加载真实的金融数据
从 CSV 文件加载真实的股市数据，并理解正确的日期时间索引。

---

1.1 使用正确的日期时间索引加载真实的股票数据
使用推荐的方法：同时使用 parse_dates 和 index_col

In [ ]:
def load_stock_data(symbol):
    """
    使用正确的日期时间索引加载股票数据。

    参数:
    symbol (str): 股票代码 (例如, 'AAPL', 'MSFT')

    返回:
    pd.DataFrame: 带有 DatetimeIndex 的 OHLCV 数据
    """
    file_path = f'./{symbol}_stock_data.csv'
    
    # 使用正确的日期时间解析和索引进行加载
    df = pd.read_csv(f"{file_path}", index_col = "Date", parse_dates = True)
    
    # 仅保留必要的 OHLCV 列
    df = df[["Open", "High", "Low", "Close", "Volume"]]
    
    return df

# 为主要科技股加载数据
symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA']
stock_data = {}

print("📊 正在加载真实的股市数据...")
print("=" * 40)

for symbol in symbols:
    try:
        stock_data[symbol] = load_stock_data(symbol)
        print(f"✅ {symbol}: 已加载 {len(stock_data[symbol])} 个交易日数据")
        print(f"   📅 Period: {stock_data[symbol].index.min().strftime('%Y-%m-%d')} to {stock_data[symbol].index.max().strftime('%Y-%m-%d')}")
    except FileNotFoundError:
        print(f"❌ {symbol}: 未找到数据文件")
        continue

print(f"\n🎯 成功加载 {len(stock_data)} 只股票")

# 显示 AAPL 的数据结构示例
print(f"\n📈 AAPL 数据示例 (前 5 行):")
print(stock_data["AAPL"].head(5))

print(f"\n📊 AAPL 数据信息:")
stock_data["AAPL"].info()

print(f"\n📏 AAPL 数据形状: {stock_data['AAPL'].shape}")
print(f"📅 索引类型: {type(stock_data['AAPL'].index)}")

# 1.2 创建一个合并的收盘价 DataFrame 以便于比较
close_prices = pd.DataFrame()
for symbol in symbols:
    if symbol in stock_data:
        close_prices[symbol] = stock_data[symbol]["Close"]

print(f"\n💰 合并的收盘价 (最近 5 天):")
print(close_prices.tail(5))
print(f"\n形状: {close_prices.shape}")
print(f"列名: {list(close_prices.columns)}")

---

## 练习 2：基础数据探索
使用基本的 pandas 方法探索我们的金融数据集的结构和特征。


---

2.1 基础的 DataFrame 探索方法

In [ ]:
print("🔍 基础数据探索")
print("=" * 40)

# 使用基本的 pandas 方法探索 AAPL 股票数据
aapl_data = stock_data["AAPL"]

print("📊 AAPL 数据概览:")
print(f"数据集形状: {aapl_data.shape}")
print(f"行数: {len(aapl_data)}")
print(f"列数: {len(aapl_data.columns)}")
print(f"列名: {list(aapl_data.columns)}")

print(f"\n📈 前 5 行 (.head()):")
print(aapl_data.head(5))

print(f"\n📉 最后 5 行 (.tail()):")
print(aapl_data.tail(5))

print(f"\n📊 基础统计信息 (.describe()):")
print(aapl_data.describe())

print(f"\n🔍 数据类型和信息 (.info()):")
aapl_data.info()

2.2 检查缺失值

In [ ]:
print(f"\n🔍 缺失值分析:")
print("每列的缺失值:")
missing_values = aapl_data.isnull().sum()
print(missing_values)

print(f"\n总缺失值: {aapl_data.isnull().sum().sum()}")
print(f"缺失数据百分比: {(aapl_data.isnull().sum().sum() / (len(aapl_data) * len(aapl_data.columns))) * 100:.2f}%")


2.3 基础数据质量检查

In [ ]:
print(f"\n✅ 数据质量检查:")

# 检查最高价 >= 最低价 (应始终为真)
high_low_check = aapl_data["High"].all() >= aapl_data["Low"].all()
print(f"最高价 >= 最低价 检查: {'✅ 通过' if high_low_check else '❌ 失败'}")

# 检查开盘价和收盘价是否在最高价和最低价之间
open_range_check = ((aapl_data['Open'] >= aapl_data['Low']) & (aapl_data['Open'] <= aapl_data['High'])).all()
close_range_check = ((aapl_data['Close'] >= aapl_data['Low']) & (aapl_data['Close'] <= aapl_data['High'])).all()

print(f"开盘价在最高价-最低价范围内: {'✅ 通过' if open_range_check else '❌ 失败'}")
print(f"收盘价在最高价-最低价范围内: {'✅ 通过' if close_range_check else '❌ 失败'}")

# 检查零值或负值
zero_negative_check = (aapl_data[['Open', 'High', 'Low', 'Close']] > 0).all().all()
print(f"无零/负价格: {'✅ 通过' if zero_negative_check else '❌ 失败'}")

2.4 日期索引探索

In [ ]:
print(f"\n📅 日期索引分析:")
print(f"开始日期: {aapl_data.index.min()}")
print(f"结束日期: {aapl_data.index.max()}")
print(f"日期范围: {(aapl_data.index.max() - aapl_data.index.min()).days} 天")
print(f"索引频率: {aapl_data.index.freq}")  # 对于不规则数据将为 None
print(f"唯一日期: {len(aapl_data.index.unique())}")
print(f"重复日期: {aapl_data.index.duplicated().sum()}")

2.5 比较所有股票的基础统计信息

In [ ]:
print(f"\n📊 所有股票价格摘要 (收盘价):")
print(close_prices.describe().round(2))

---

## 练习 3：基于时间的索引和切片
学习使用日期时间索引来过滤和切片金融数据 - 这是金融分析的关键技能。

---

3.1 使用布尔索引进行基于时间的过滤

In [ ]:
print("📅 基于时间的索引和切片")
print("=" * 40)

# 使用 AAPL 数据进行演示
aapl = stock_data['AAPL']

# 方法 1：使用日期时间组件的布尔索引
print("📊 方法 1：使用日期时间组件的布尔索引")

# 过滤特定年份的数据
data_2024 = aapl[aapl.index.year == 2024]
print(f"2024 年数据: {len(data_2024)} 个交易日")
print(data_2024.head())

# 过滤特定月份和年份的数据
data_jan_2024 = aapl[(aapl.index.year == 2024) & (aapl.index.month == 1)]
print(f"\n2024 年 1 月数据: {len(data_jan_2024)} 个交易日")
print(data_jan_2024.head())

# 过滤多个年份的数据
data_recent = aapl[aapl.index.year >= 2023]
print(f"\n2023 年至今的数据: {len(data_recent)} 个交易日")

# 方法 2：使用 .loc 进行日期范围切片
print(f"\n📊 方法 2：使用 .loc 进行日期范围切片")

# 使用完整日期字符串切片
jan_2024_slice = aapl.loc["2024-01-01":"2024-01-31"]
print(f"使用 .loc 切片的一月 2024: {len(jan_2024_slice)} 天")
print(jan_2024_slice.head())

# 为特定季度切片
q1_2024 = aapl.loc["2024-01-01":"2024-03-31"]
print(f"\n2024 年第一季度: {len(q1_2024)} 个交易日")

# 最近 30 个交易日的切片
last_30_days = aapl.tail(30)
print(f"\n最近 30 个交易日:")
print(last_30_days[['Close']].head())

# 3.2 高级的基于时间的操作
print(f"\n📈 高级基于时间的操作:")

# 按年份分组并计算基础统计信息
yearly_stats = aapl.groupby(aapl.index.year)['Close'].agg(['first', 'last', 'min', 'max', 'mean'])

yearly_stats.columns = ['首日价格', '最后价格', '最低价格', '最高价格', '平均价格']
yearly_stats['年度回报_%'] = ((yearly_stats['最后价格'] - yearly_stats['首日价格']) / yearly_stats['首日价格'] * 100).round(2)

print("年度统计信息:")
print(yearly_stats)

# 2024 年按月分组
monthly_2024 = data_2024.groupby(data_2024.index.month)['Close'].agg(['mean', 'min', 'max'])
monthly_2024.index = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月'][:len(monthly_2024)]
print(f"\n2024 年月度统计信息:")
print(monthly_2024.round(2))

# 3.3 星期几分析
print(f"\n📊 星期几分析:")

# 添加星期几信息
aapl_with_weekday = aapl.copy()
aapl_with_weekday['Weekday'] = aapl_with_weekday.index.day_name()
aapl_with_weekday['Returns'] = aapl_with_weekday["Close"].pct_change() * 100

# 按星期几计算平均收益率
weekday_analysis = aapl_with_weekday.groupby("Weekday").agg({
    "Close": "mean",
    "Returns": "mean"
}).round(2)
print("按星期几的平均表现:")
print(weekday_analysis)

# 显示交易日频率 (股市数据应为周一至周五)
weekday_counts = aapl_with_weekday["Weekday"].value_counts()
print(f"\n交易日频率:")
print(weekday_counts)

---

## 练习 4：基础金融计算
计算基础金融指标：收益率、移动平均线和简单的波动率度量。

---

In [ ]:
# 4.1 计算简单的日收益率
print("💹 基础金融计算")
print("=" * 40)

# 使用 AAPL 数据进行演示
aapl = stock_data['AAPL']

# 使用 .pct_change() 计算每日百分比收益率
aapl_returns = aapl["Close"].pct_change() * 100
print("📊 日收益率计算:")
print("前 10 个日收益率 (%):")
print(aapl_returns.head(10).round(4))

print(f"\n收益率统计信息:")
print(aapl_returns.describe().round(4))

# 计算所有股票的收益率
print(f"\n📈 所有股票的收益率:")
daily_returns = close_prices.pct_change() * 100
print("平均日收益率 (%):")
print(daily_returns.mean().round(4))

print(f"\n日波动率 (收益率的标准差):")
print(daily_returns.std().round(4))

# 4.2 简单移动平均线
print(f"\n📊 简单移动平均线:")

# 为 AAPL 计算不同的移动平均线
aapl_sma_5 = aapl["Close"].rolling(window = 5).mean()
aapl_sma_20 = aapl["Close"].rolling(window = 20).mean()
aapl_sma_50 = aapl["Close"].rolling(window = 50).mean()

# 创建一个摘要 DataFrame
ma_summary = pd.DataFrame({
    'Close': aapl["Close"],
    'SMA_5': aapl_sma_5,
    'SMA_20': aapl_sma_20,
    'SMA_50': aapl_sma_50
})

print("AAPL 移动平均线 (最近 10 天):")
print(ma_summary.tail(10).round(2))

# 4.3 基础波动率计算
print(f"\n📊 基础波动率度量:")

# 滚动波动率 (20 天窗口)
aapl_vol_20 = aapl_returns.rolling(window = 20).std()
print("20 日滚动波动率 (最近 10 个值):")
print(aapl_vol_20.tail(10).round(4))

# 计算所有股票的波动率
all_volatility = daily_returns.rolling(window = 20).std()
print(f"\n所有股票的当前 20 日波动率:")
print(all_volatility.tail(1).round(4))

# 4.4 简单的相关性分析
print(f"\n🔗 基础相关性分析:")

# 计算收盘价的相关性矩阵
price_correlation = close_prices.corr()
print("相关性矩阵 (收盘价):")
print(price_correlation.round(4))

# 计算收益率的相关性矩阵
return_correlation = daily_returns.corr()
print(f"\n相关性矩阵 (日收益率):")
print(return_correlation.round(4))

# 4.5 计算累积收益率
print(f"\n📈 累积收益率:")

# 计算累积收益率 (复合增长)
cumulative_returns = (1 + daily_returns / 100).cumprod() - 1
print("总累积收益率 (%):")
print((cumulative_returns.tail(1) * 100).round(2))

# 显示 $10,000 的投资增长
initial_investment = 10000
final_values = initial_investment * (1 + cumulative_returns.tail(1))
print(f"\n$10,000 投资的增长:")
for symbol in final_values.columns:
    print(f"{symbol}: ${final_values[symbol].iloc[0]:,.2f}")

# 4.6 价格区间和表现指标
print(f"\n📏 价格区间分析:")

# 计算 52 周高点和低点 (使用可用数据近似)
price_ranges = pd.DataFrame()
for symbol in close_prices.columns:
    recent_data = close_prices[symbol].tail(252)
    price_ranges[symbol] = [
        recent_data.min(),
        recent_data.max(),
        recent_data.iloc[-1],  # 当前价格
        ((recent_data.iloc[-1] - recent_data.min()) / recent_data.min() * 100),  # 距低点百分比
        ((recent_data.max() - recent_data.iloc[-1]) / recent_data.max() * 100)   # 距高点百分比
    ]

price_ranges.index = ['52周低点', '52周高点', '当前', '距低点_%', '距高点_%']
print("52 周表现摘要:")
print(price_ranges.round(2))

---

## 练习 5：时间序列操作
学习基础的时间序列操作：重采样、滚动窗口和位移。

---

In [ ]:
# 5.1 基础重采样操作
print("⏰ 时间序列操作")
print("=" * 40)

# 使用 AAPL 数据进行演示
aapl = stock_data['AAPL']

# 重采样为周数据 (使用 'W' 而不是 'W-FRI')
print("📊 重采样到不同频率:")

# 周数据 - 取每周的最后价格
weekly_prices = aapl['Close'].resample('W').last()
print(f"周价格 (最近 10 周):")
print(weekly_prices.tail(10).round(2))

# 月数据 - 取每月的最后价格
monthly_prices = aapl.resample("M").last()
print(f"\n月价格 (最近 6 个月):")
print(monthly_prices.tail(6).round(2))

# 周摘要统计信息
weekly_summary = aapl.resample('W').agg({
    'Open': 'first',    # 本周第一个价格
    'High': 'max',      # 本周最高价格
    'Low': 'min',       # 本周最低价格
    'Close': 'last',    # 本周最后一个价格
    'Volume': 'sum'     # 本周总成交量
})

print(f"\n周 OHLCV 摘要 (最近 5 周):")
print(weekly_summary.tail(5).round(2))

# 5.2 滚动窗口操作
print(f"\n📊 滚动窗口操作:")

# 基础滚动统计信息
rolling_stats = pd.DataFrame({
    'Price': aapl['Close'],
    'SMA_10': aapl["Close"].rolling(window = 10).mean(),      # 10 日简单移动平均线
    'SMA_30': aapl["Close"].rolling(window = 30).mean(),      # 30 日简单移动平均线
    'Rolling_Min_20': aapl["Close"].rolling(window = 20).min(),   # 20 日滚动最小值
    'Rolling_Max_20': aapl["Close"].rolling(window = 20).max(),   # 20 日滚动最大值
    'Rolling_Std_20': aapl["Close"].rolling(window = 20).std()    # 20 日滚动标准差
})

print("滚动统计信息 (最近 10 天):")
print(rolling_stats.tail(10).round(2))

# 滚动成交量分析
volume_stats = pd.DataFrame({
    'Volume': aapl['Volume'],
    'Avg_Volume_20': aapl["Volume"].rolling(window = 20).mean(),     # 20 日平均成交量
    'Volume_Ratio': aapl["Volume"] / aapl["Volume"].rolling(window = 20).mean()       # 当前成交量 / 20 日平均成交量
})

print(f"\n成交量分析 (最近 10 天):")
print(volume_stats.tail(10).round(2))

# 5.3 扩展窗口操作
print(f"\n📈 扩展窗口操作:")

# 扩展统计信息 (从开始累积)
expanding_stats = pd.DataFrame({
    'Price': aapl['Close'],
    'Expanding_Mean': aapl['Close'].expanding().mean(),    # 扩展均值
    'Expanding_Min': aapl['Close'].expanding().min(),     # 扩展最小值
    'Expanding_Max': aapl['Close'].expanding().max(),     # 扩展最大值
    'Expanding_Std': aapl['Close'].expanding().std()      # 扩展标准差
})

print("扩展统计信息 (最近 10 天):")
print(expanding_stats.tail(10).round(2))

# 5.4 用于滞后分析的位移操作
print(f"\n🔄 位移操作:")

# 创建滞后变量
lagged_data = pd.DataFrame({
    'Close': aapl['Close'],
    'Close_Lag1': aapl['Close'].shift(1),        # 前一天使用 .shift(1)
    'Close_Lag5': aapl['Close'].shift(5),        # 5 天前使用 .shift(5)
    'Close_Lead1': aapl['Close'].shift(-1)        # 下一天使用 .shift(-1)
})

# 计算价格变化
lagged_data['Daily_Change'] = lagged_data["Close"] - lagged_data["Close_Lag1"]
lagged_data['5Day_Change'] = lagged_data["Close"] - lagged_data["Close_Lag5"]

print("滞后价格分析 (显示第 10-15 天):")
print(lagged_data.iloc[10:15].round(2))

# 5.5 多股票重采样
print(f"\n📊 多股票重采样:")

# 将所有股票重采样为月频率
monthly_closes = close_prices.resample("M").last()
print("月收盘价 (最近 6 个月):")
print(monthly_closes.tail(6).round(2))

# 计算月收益率
monthly_returns = monthly_closes.pct_change() * 100
print(f"\n月收益率 (%):")
print(monthly_returns.tail(6).round(2))

# 5.6 工作日操作
print(f"\n📅 工作日分析:")

# 统计 2024 年每月的交易日数
if len(aapl[aapl.index.year == 2024]) > 0:
    trading_days_2024 = aapl[aapl.index.year == 2024].groupby(aapl[aapl.index.year == 2024].index.month).size()
    trading_days_2024.index = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月'][:len(trading_days_2024)]
    print("2024 年每月交易日数:")
    print(trading_days_2024)

# 按月计算平均日成交量
avg_volume_monthly = aapl.groupby(aapl.index.month)["Volume"].mean()
avg_volume_monthly.index = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月']
print(f"\n月平均日成交量:")
print((avg_volume_monthly / 1000000).round(2))  # 转换为百万

---

## 练习 6：缺失数据处理
学习识别和处理金融时间序列数据中的缺失值。

---

In [ ]:
# 6.1 识别缺失数据
print("🔍 缺失数据处理")
print("=" * 40)

# 检查数据集中的缺失值
print("📊 缺失数据评估:")

# 单独检查每只股票
for symbol in symbols:
    if symbol in stock_data:
        missing_count = stock_data[symbol].isnull().sum().sum()
        total_cells = len(stock_data[symbol]) * len(stock_data[symbol].columns)
        missing_pct = (missing_count / total_cells) * 100
        print(f"{symbol}: {missing_count} 个缺失值 ({missing_pct:.2f}%)")

# 检查合并的收盘价
print(f"\n合并收盘价的缺失数据:")
print(close_prices.isnull().sum())

# 6.2 创建带有缺失值的示例数据用于演示
print(f"\n🛠️ 缺失数据处理技术:")

# 创建 AAPL 数据的副本，并引入一些缺失值用于演示
demo_data = stock_data['AAPL']['Close'].copy()

# 在特定位置引入缺失值
demo_data.iloc[100:103] = np.nan  # 移除连续 3 天
demo_data.iloc[200] = np.nan      # 移除单天
demo_data.iloc[250:252] = np.nan  # 移除连续 2 天

print(f"创建的带有缺失值的演示数据:")
print(f"原始数据点: {len(stock_data['AAPL'])}")
print(f"引入的缺失值: {demo_data.isnull().sum()}")

# 显示缺失数据区域
print(f"\n位置 100 附近的缺失数据:")
print(demo_data.iloc[98:105])

# 6.3 前向填充方法
print(f"\n📈 前向填充方法:")
demo_ffill = demo_data.fillna(method = "ffill")
print("前向填充 - 将最后一个已知值向前传递:")
print(demo_ffill.iloc[98:105].round(2))

# 6.4 后向填充方法
print(f"\n📉 后向填充方法:")
demo_bfill = demo_data.fillna(method = "bfill")
print("后向填充 - 使用下一个已知值:")
print(demo_bfill.iloc[98:105].round(2))

# 6.5 线性插值
print(f"\n📊 线性插值方法:")
demo_interpolate = demo_data.interpolate(method = "linear")
print("线性插值 - 在已知点之间估计值:")
print(demo_interpolate.iloc[98:105].round(2))

# 6.6 比较不同方法
print(f"\n🔍 方法比较:")
comparison_df = pd.DataFrame({
    'Original': demo_data.iloc[98:105],
    'Forward_Fill': demo_ffill.iloc[98:105],
    'Backward_Fill': demo_bfill.iloc[98:105],
    'Interpolation': demo_interpolate.iloc[98:105]
})

print(comparison_df.round(2))

# 6.7 处理收益率计算中的缺失数据
print(f"\n💹 收益率计算中的缺失数据:")

# 使用缺失数据计算收益率
demo_returns = demo_data.pct_change() * 100
print(f"带有缺失数据的收益率 (位置 100 附近):")
print(demo_returns.iloc[98:105].round(4))

# 前向填充后计算收益率
demo_returns_filled = demo_ffill.pct_change() * 100
print(f"\n前向填充后的收益率:")
print(demo_returns_filled.iloc[98:105].round(4))

# 6.8 金融数据的最佳实践
print(f"\n✅ 金融缺失数据的最佳实践:")

print("1. 前向填充: 价格数据最常用")
print("   - 假设最后一个已知价格持续到有新信息")
print("   - 对价格数据的保守方法")

print(f"\n2. 插值: 适用于平滑数据")
print("   - 基于趋势估计缺失值")
print("   - 在波动市场中谨慎使用")

print(f"\n3. 删除缺失值:")
# 显示删除缺失值的效果
demo_dropped = demo_data.dropna()
print(f"   - 原始长度: {len(demo_data)}")
print(f"   - 删除 NaN 后: {len(demo_dropped)}")
print(f"   - 丢失的数据点: {len(demo_data) - len(demo_dropped)}")

# 6.9 处理多股票分析中的缺失数据
print(f"\n📊 多股票缺失数据:")

# 检查所有股票之间的日期对齐情况
common_dates = close_prices.dropna().index
print(f"所有股票都存在的日期: {len(common_dates)}")
print(f"总交易日数: {len(close_prices)}")
print(f"有任何缺失数据的天数: {len(close_prices) - len(common_dates)}")

# 前向填充所有股票
close_prices_filled = close_prices.fillna(method = "ffill")
print(f"\n前向填充后的缺失值:")
print(close_prices_filled.isnull().sum())

# 比较处理缺失数据前后的相关性
print(f"\n🔗 缺失数据对相关性的影响:")
corr_with_missing = close_prices.corr()
corr_filled = close_prices_filled.corr()

print("相关性差异 (AAPL vs MSFT):")
print(f"有缺失数据时: {corr_with_missing.loc['AAPL', 'MSFT']:.4f}")
print(f"前向填充后: {corr_filled.loc['AAPL', 'MSFT']:.4f}")
print(f"差异: {abs(corr_with_missing.loc['AAPL', 'MSFT'] - corr_filled.loc['AAPL', 'MSFT']):.4f}")

---

## 练习 7：基础分组和聚合

---

In [ ]:
# 7.1 按时间段分组
print("📊 基础分组和聚合")
print("=" * 40)

# 使用 AAPL 数据进行演示
aapl = stock_data['AAPL']

# 按年份分组并计算基础统计信息
print("📅 年度分析:")
yearly_stats = aapl.groupby(aapl.index.year).agg({
    'Open': 'first',      # 第一个交易日的价格
    'Close': 'last',      # 最后一个交易日的价格
    'High': 'max',        # 年度最高价格
    'Low': 'min',         # 年度最低价格
    'Volume': 'mean'      # 平均日成交量
}).round(2)

# 添加年度收益率计算
yearly_stats['Return_%'] = ((yearly_stats["Close"] - yearly_stats["Open"]) / yearly_stats["Open"] * 100).round(2)

print("AAPL 年度统计信息:")
print(yearly_stats)

# 按月分组 (近期数据)
print(f"\n📅 月度分析 (2024):")
if len(aapl[aapl.index.year == 2024]) > 0:
    monthly_2024 = aapl[aapl.index.year == 2024].groupby(aapl[aapl.index.year == 2024].index.month).agg({
        'Close': ['first', 'last', 'min', 'max', 'mean'],
        'Volume': 'mean'
    }).round(2)

    # 扁平化列名
    monthly_2024.columns = ['_'.join(col).strip() for col in monthly_2024.columns]
    monthly_2024.index = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月'][:len(monthly_2024)]

    print(monthly_2024)

# 7.2 按星期几分组
print(f"\n📅 星期几分析:")
aapl_copy = aapl.copy()
aapl_copy['Weekday'] = aapl_copy.index.day_name()
aapl_copy['Returns'] = aapl_copy["Close"].pct_change() * 100

weekday_analysis = aapl_copy.groupby('Weekday').agg({
    'Close': 'mean',
    'Returns': ['mean', 'std'],
    'Volume': 'mean'
}).round(4)

print("按星期几的平均表现:")
print(weekday_analysis)

# 7.3 多股票比较
print(f"\n📊 多股票比较:")

# 计算所有股票的基础指标
stock_comparison = pd.DataFrame()
for symbol in symbols:
    if symbol in stock_data:
        stock = stock_data[symbol]
        returns = stock["Close"].pct_change() * 100

        stock_comparison[symbol] = [
            stock['Close'].iloc[-1],                    # 当前价格
            stock['Close'].mean(),                      # 平均价格
            returns.mean(),                             # 平均收益率
            returns.std(),                              # 波动率
            stock['Volume'].mean() / 1000000,           # 平均成交量 (百万)
            (stock['Close'].iloc[-1] / stock['Close'].iloc[0] - 1) * 100  # 总收益率
        ]

stock_comparison.index = ['当前价格', '平均价格', '平均收益率_%', '波动率_%', '平均成交量_百万', '总收益率_%']
print("股票比较摘要:")
print(stock_comparison.round(2))

# 7.4 表现排名
print(f"\n🏆 表现排名:")

# 按不同指标对股票进行排名
rankings = pd.DataFrame({
    '总收益率_%': stock_comparison.loc["总收益率_%"].rank(ascending = False),        # 按总收益率排名 (降序)
    '平均收益率_%': stock_comparison.loc["平均收益率_%"].rank(ascending = False),          # 按平均收益率排名 (降序)
    '波动率_%': stock_comparison.loc["波动率_%"].rank(ascending = True),          # 按波动率排名 (升序 - 越低越好)
    '成交量排名': stock_comparison.loc["平均成交量_百万"].rank(ascending = False)            # 按成交量排名 (降序)
})

print("股票排名 (1 = 最好):")
print(rankings.astype(int))

# 7.5 计算投资组合统计信息
print(f"\n💼 简单投资组合分析:")

# 等权重投资组合
equal_weights = [0.25, 0.25, 0.25, 0.25]  # 4 只股票的等权重
portfolio_returns = (close_prices.pct_change() * equal_weights).sum(axis = 1) * 100

portfolio_stats = {
    '投资组合平均收益率_%': portfolio_returns.mean(),
    '投资组合波动率_%': portfolio_returns.std(),
    '投资组合总收益率_%': ((close_prices.iloc[-1] * equal_weights).sum() / (close_prices.iloc[0] * equal_weights).sum() - 1) * 100
}

print("等权重投资组合表现:")
for key, value in portfolio_stats.items():
    print(f"{key}: {value:.4f}")

# 比较投资组合与个股
print(f"\n📊 投资组合 vs 个股:")
comparison_table = pd.DataFrame({
    '平均收益率_%': [portfolio_stats['投资组合平均收益率_%']] + [stock_comparison.loc['平均收益率_%', symbol] for symbol in symbols],
    '波动率_%': [portfolio_stats['投资组合波动率_%']] + [stock_comparison.loc['波动率_%', symbol] for symbol in symbols],
    '总收益率_%': [portfolio_stats['投资组合总收益率_%']] + [stock_comparison.loc['总收益率_%', symbol] for symbol in symbols]
}, index=['投资组合'] + symbols)

print(comparison_table.round(2))

---

## 练习 8：数据导出和文件操作
学习以各种格式保存分析结果以供进一步使用。

---

In [ ]:
# 8.1 将数据导出到 CSV 文件
print("💾 数据导出和文件操作")
print("=" * 40)

# 导出单个股票数据
print("📁 正在导出单个数据集:")
for symbol in symbols:
    if symbol in stock_data:
        filename = f'./Lab2_{symbol}_analysis.csv'

        # 为每只股票创建分析摘要
        stock = stock_data[symbol]
        returns = stock["Close"].pct_change() * 100

        analysis_df = pd.DataFrame({
            'Date': stock.index,
            'Close': stock['Close'],
            'SMA_20': stock["Close"].rolling(window = 20).mean(),         # 20 日移动平均线
            'SMA_50': stock["Close"].rolling(window = 50).mean(),         # 50 日移动平均线
            'Daily_Return_%': returns,
            'Rolling_Vol_20': returns.rolling(window = 20).std(), # 20 日滚动波动率
            'Volume_M': stock['Volume'] / 1000000
        })

        analysis_df.to_csv(filename, index=False)
        print(f"✅ {symbol}: {filename} ({len(analysis_df)} 行)")

# 导出合并的收盘价
close_prices.to_csv('Lab2_close_prices.csv')
print(f"✅ 合并收盘价: Lab2_close_prices.csv")

# 导出日收益率
daily_returns = close_prices.pct_change() * 100
daily_returns.to_csv('Lab2_daily_returns.csv')
print(f"✅ 日收益率: Lab2_daily_returns.csv")

# 8.2 导出到带多个工作表的 Excel
print(f"\n📊 正在创建全面的 Excel 工作簿:")

with pd.ExcelWriter('Lab2_Stock_Analysis.xlsx', engine='openpyxl') as writer:
    # 工作表 1: 收盘价
    close_prices.to_excel(writer, sheet_name='Close_Prices')

    # 工作表 2: 日收益率
    daily_returns.to_excel(writer, sheet_name='Daily_Returns')

    # 工作表 3: 相关性矩阵
    close_prices.corr().to_excel(writer, sheet_name='Correlations')

    # 工作表 4: 摘要统计信息
    summary_stats = pd.DataFrame({
        'Avg_Price': close_prices.mean(),
        'Std_Price': close_prices.std(),
        'Min_Price': close_prices.min(),
        'Max_Price': close_prices.max(),
        'Avg_Return_%': daily_returns.mean(),
        'Volatility_%': daily_returns.std(),
        'Total_Return_%': (close_prices.iloc[-1] / close_prices.iloc[0] - 1) * 100
    })
    summary_stats.to_excel(writer, sheet_name='Summary_Stats')

print(f"✅ Excel 工作簿: Lab2_Stock_Analysis.xlsx (4 个工作表)")

# 8.3 创建简单的摘要报告
print(f"\n📝 正在创建摘要报告:")

# 计算关键指标

# 找到最佳和最差表现者
best_performer = (close_prices.iloc[-1] / close_prices.iloc[0] - 1).idxmax()
worst_performer = (close_prices.iloc[-1] / close_prices.iloc[0] - 1).idxmin()

# 找到最低和最高波动率的股票
lowest_volatility = daily_returns.std().idxmin()
highest_volatility = daily_returns.std().idxmax()

# 计算总收益率
total_returns = (close_prices.iloc[-1] / close_prices.iloc[0] - 1) * 100

# 使用关键发现创建摘要报告字符串
summary_report = f"""
基础 Pandas 金融分析报告
=====================================

分析期间: {close_prices.index.min().strftime('%Y-%m-%d')} 至 {close_prices.index.max().strftime('%Y-%m-%d')}
交易日数: {len(close_prices)}
分析的股票: {', '.join(close_prices.columns)}

表现摘要:
-------------------
最佳表现者: {best_performer} ({total_returns[best_performer]:.2f}%)
最差表现者: {worst_performer} ({total_returns[worst_performer]:.2f}%)
最低波动率: {lowest_volatility} ({daily_returns.std()[lowest_volatility]:.2f}%)
最高波动率: {highest_volatility} ({daily_returns.std()[highest_volatility]:.2f}%)

个股表现:
-----------------------------
"""

for symbol in close_prices.columns:
    avg_return = daily_returns[symbol].mean()
    volatility = daily_returns[symbol].std()
    total_return = total_returns[symbol]

    summary_report += f"""
{symbol}:
  - 总收益率: {total_return:.2f}%
  - 平均日收益率: {avg_return:.4f}%
  - 波动率 (日): {volatility:.4f}%
  - 当前价格: ${close_prices[symbol].iloc[-1]:.2f}
"""

summary_report += f"""

相关性分析:
--------------------
最高相关性: {close_prices.corr().unstack().drop_duplicates().sort_values(ascending=False).iloc[1]:.4f}
平均相关性: {close_prices.corr().unstack().drop_duplicates().mean():.4f}

数据质量:
------------
缺失值: {close_prices.isnull().sum().sum()}
数据完整性: {((len(close_prices) * len(close_prices.columns) - close_prices.isnull().sum().sum()) / (len(close_prices) * len(close_prices.columns))) * 100:.1f}%

创建的文件:
--------------
- Lab2_close_prices.csv
- Lab2_daily_returns.csv
- Lab2_Stock_Analysis.xlsx
- 每只股票的单独分析文件

使用基础 Pandas 进行金融数据分析生成
日期: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# 保存报告
with open('Lab2_Analysis_Report.txt', 'w') as f:
    f.write(summary_report)

print(f"✅ 摘要报告: Lab2_Analysis_Report.txt")

# 8.4 显示文件摘要
print(f"\n📋 创建的文件摘要:")
print("=" * 30)
print("1. 单个股票 CSV 文件")
print("2. Lab2_close_prices.csv - 合并收盘价")
print("3. Lab2_daily_returns.csv - 日收益率百分比") 
print("4. Lab2_Stock_Analysis.xlsx - 全面的 Excel 工作簿")
print("5. Lab2_Analysis_Report.txt - 摘要分析报告")

print(f"\n✅ 所有数据导出操作成功完成!")
print(f"🎯 你现在已拥有全面的数据集，可供进一步分析或演示。")

print(summary_report)